In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import random

In [3]:
train = pd.read_csv("train.csv")

In [4]:
train_1JHC = train[train["type"] == '1JHC']

In [5]:
train_1JHC.shape

(709416, 6)

In [6]:
structures = pd.read_csv("structures.csv")

In [7]:
inter_train_1JHC = train_1JHC.merge(structures, how='inner', left_on=['molecule_name','atom_index_0'], 
                right_on=['molecule_name','atom_index'])
loaded_train_1JHC = inter_train_1JHC.merge(structures, how='inner', left_on=['molecule_name','atom_index_1'], 
                right_on=['molecule_name','atom_index'])

In [9]:
loaded_train_1JHC[:20]

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_index_x,atom_x,x_x,y_x,z_x,atom_index_y,atom_y,x_y,y_y,z_y
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,1,H,0.002150,-0.006031,0.001976,0,C,-0.012698,1.085804,0.008001
1,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,2,H,1.011731,1.463751,0.000277,0,C,-0.012698,1.085804,0.008001
2,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,3,H,-0.540815,1.447527,-0.876644,0,C,-0.012698,1.085804,0.008001
3,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,4,H,-0.523814,1.437933,0.906397,0,C,-0.012698,1.085804,0.008001
4,17,dsgdb9nsd_000005,2,0,1JHC,171.2200,2,H,-0.027803,2.198949,0.014154,0,C,-0.013324,1.132466,0.008276
5,19,dsgdb9nsd_000007,2,0,1JHC,83.5429,2,H,0.994873,1.939743,0.002941,0,C,-0.018704,1.525582,0.010433
6,26,dsgdb9nsd_000007,3,0,1JHC,83.5417,3,H,-0.542076,1.923611,-0.865117,0,C,-0.018704,1.525582,0.010433
7,32,dsgdb9nsd_000007,4,0,1JHC,83.5484,4,H,-0.525241,1.914173,0.900024,0,C,-0.018704,1.525582,0.010433
8,38,dsgdb9nsd_000007,5,1,1JHC,83.5418,5,H,0.525487,-0.401908,0.877544,1,C,0.002104,-0.003882,0.001999
9,42,dsgdb9nsd_000007,6,1,1JHC,83.5430,6,H,-1.011477,-0.418034,0.009508,1,C,0.002104,-0.003882,0.001999


In [15]:
loaded_train_1JHC_dropped_cols = loaded_train_1JHC.drop(['atom_index_x','atom_index_y','atom_x','atom_y','type','id','molecule_name','atom_index_0','atom_index_1'], 
                                                        axis=1)

In [16]:
loaded_train_1JHC_dropped_cols[:10]
    

,scalar_coupling_constant,x_x,y_x,z_x,x_y,y_y,z_y
0,84.8076,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001
1,84.8074,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001
2,84.8093,-0.540815,1.447527,-0.876644,-0.012698,1.085804,0.008001
3,84.8095,-0.523814,1.437933,0.906397,-0.012698,1.085804,0.008001
4,171.2200,-0.027803,2.198949,0.014154,-0.013324,1.132466,0.008276
5,83.5429,0.994873,1.939743,0.002941,-0.018704,1.525582,0.010433
6,83.5417,-0.542076,1.923611,-0.865117,-0.018704,1.525582,0.010433
7,83.5484,-0.525241,1.914173,0.900024,-0.018704,1.525582,0.010433
8,83.5418,0.525487,-0.401908,0.877544,0.002104,-0.003882,0.001999
9,83.5430,-1.011477,-0.418034,0.009508,0.002104,-0.003882,0.001999


In [17]:
loaded_train_1JHC_dropped_cols.shape

(709416, 7)

In [18]:
train_1JHC_array = loaded_train_1JHC_dropped_cols.as_matrix()

/Users/ankushhore/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [19]:
train_1JHC_array.shape

(709416, 7)

In [20]:
np.random.shuffle(train_1JHC_array)

In [22]:
train_1JHC_array_train = train_1JHC_array[:500000]
train_1JHC_array_val = train_1JHC_array[500000:]

In [30]:
X_train = train_1JHC_array_train[:,1:]
y_train = train_1JHC_array_train[:,0].reshape((500000,1))
X_val = train_1JHC_array_val[:,1:]
y_val = train_1JHC_array_val[:,0].reshape((209416,1))

In [31]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(500000, 6) (500000, 1) (209416, 6) (209416, 1)


In [35]:
X = tf.placeholder(tf.float32, shape=[None, 6])
y = tf.placeholder(tf.float32)
training_flag = 1
batch_size = tf.placeholder(tf.int64)
dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size).repeat()
iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

In [36]:
with tf.name_scope("cnn"):
    X1 = tf.layers.dense(features, 80, activation = 'relu')
    X2 = tf.layers.dropout(X1, rate=0.5)
    X3 = tf.layers.dense(X2, 40, activation = 'relu')
    X4 = tf.layers.dropout(X3, rate=0.5)
    X5 = tf.layers.dense(X4, 20, activation = 'relu')
    X6 = tf.layers.dropout(X5, rate=0.5)
    X7 = tf.layers.dense(X6, 10, activation = 'relu')
    X8 = tf.layers.dropout(X7, rate=0.5)
    output = tf.layers.dense(X8, 1, activation = None)

In [37]:
with tf.name_scope("loss"):
    l2_loss = tf.losses.mean_squared_error(labels, output)

In [38]:
learning_rate = 0.01
with tf.name_scope("train"):
    training_op = tf.train.AdamOptimizer().minimize(l2_loss)
    #training_op = optimizer.minimize(l2_loss)

Instructions for updating:
Use tf.cast instead.


In [39]:
with tf.name_scope("estimated_MAE"):
    MAE_hat = tf.math.reduce_mean(tf.metrics.mean_absolute_error(labels, output))
    log_MAE_hat = tf.math.log(MAE_hat)

In [40]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [44]:
EPOCHS = 500
BATCH_SIZE = len(X_train)
n_batches = 1

In [45]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    # initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ X: X_train, y: y_train, batch_size: BATCH_SIZE})
    print('Training...')
    for i in range(EPOCHS):
        tot_loss = 0
        for _ in range(n_batches):
            _, loss_value, tr_lmae = sess.run([training_op, l2_loss, log_MAE_hat])
            tot_loss += loss_value
        print("Iter: {}, Loss: {:.4f}, Train MAE hat: {:.4f}".format(i, tot_loss / n_batches, tr_lmae))
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ X: X_val, y: y_val, batch_size: X_val.shape[0]})
    t_loss, l_mae = sess.run([l2_loss, log_MAE_hat])
    print('Test Loss: {:.4f}, Test MAE hat: {:.4f}'.format(t_loss, l_mae))
    save_path = saver.save(sess, "./my_model_final4.ckpt")

Training...
Iter: 0, Loss: 9375.6572, Train MAE hat: 3.8616
Iter: 1, Loss: 9363.0010, Train MAE hat: 4.5546
Iter: 2, Loss: 9351.1475, Train MAE hat: 4.5543
Iter: 3, Loss: 9340.1396, Train MAE hat: 4.5539
Iter: 4, Loss: 9330.1289, Train MAE hat: 4.5536
Iter: 5, Loss: 9320.8945, Train MAE hat: 4.5533
Iter: 6, Loss: 9312.0166, Train MAE hat: 4.5531
Iter: 7, Loss: 9303.2217, Train MAE hat: 4.5528
Iter: 8, Loss: 9294.4258, Train MAE hat: 4.5525
Iter: 9, Loss: 9285.5469, Train MAE hat: 4.5523
Iter: 10, Loss: 9276.4893, Train MAE hat: 4.5520
Iter: 11, Loss: 9267.1689, Train MAE hat: 4.5518
Iter: 12, Loss: 9257.4697, Train MAE hat: 4.5515
Iter: 13, Loss: 9247.2461, Train MAE hat: 4.5513
Iter: 14, Loss: 9236.3838, Train MAE hat: 4.5510
Iter: 15, Loss: 9224.7627, Train MAE hat: 4.5507
Iter: 16, Loss: 9212.2412, Train MAE hat: 4.5504
Iter: 17, Loss: 9198.7188, Train MAE hat: 4.5502
Iter: 18, Loss: 9184.0938, Train MAE hat: 4.5499
Iter: 19, Loss: 9168.2881, Train MAE hat: 4.5496
Iter: 20, Loss: 91

Iter: 167, Loss: 915.3192, Train MAE hat: 3.9666
Iter: 168, Loss: 912.4571, Train MAE hat: 3.9633
Iter: 169, Loss: 909.5982, Train MAE hat: 3.9601
Iter: 170, Loss: 906.7426, Train MAE hat: 3.9568
Iter: 171, Loss: 903.8891, Train MAE hat: 3.9536
Iter: 172, Loss: 901.0373, Train MAE hat: 3.9504
Iter: 173, Loss: 898.1863, Train MAE hat: 3.9472
Iter: 174, Loss: 895.3358, Train MAE hat: 3.9441
Iter: 175, Loss: 892.4851, Train MAE hat: 3.9409
Iter: 176, Loss: 889.6337, Train MAE hat: 3.9378
Iter: 177, Loss: 886.7809, Train MAE hat: 3.9347
Iter: 178, Loss: 883.9268, Train MAE hat: 3.9317
Iter: 179, Loss: 881.0710, Train MAE hat: 3.9286
Iter: 180, Loss: 878.2135, Train MAE hat: 3.9256
Iter: 181, Loss: 875.3542, Train MAE hat: 3.9226
Iter: 182, Loss: 872.4937, Train MAE hat: 3.9196
Iter: 183, Loss: 869.6315, Train MAE hat: 3.9166
Iter: 184, Loss: 866.7679, Train MAE hat: 3.9136
Iter: 185, Loss: 863.9027, Train MAE hat: 3.9107
Iter: 186, Loss: 861.0358, Train MAE hat: 3.9078
Iter: 187, Loss: 858

Iter: 335, Loss: 452.4472, Train MAE hat: 3.5869
Iter: 336, Loss: 450.4379, Train MAE hat: 3.5852
Iter: 337, Loss: 448.4464, Train MAE hat: 3.5834
Iter: 338, Loss: 446.4727, Train MAE hat: 3.5817
Iter: 339, Loss: 444.5171, Train MAE hat: 3.5800
Iter: 340, Loss: 442.5793, Train MAE hat: 3.5783
Iter: 341, Loss: 440.6595, Train MAE hat: 3.5766
Iter: 342, Loss: 438.7573, Train MAE hat: 3.5749
Iter: 343, Loss: 436.8733, Train MAE hat: 3.5732
Iter: 344, Loss: 435.0071, Train MAE hat: 3.5715
Iter: 345, Loss: 433.1592, Train MAE hat: 3.5698
Iter: 346, Loss: 431.3293, Train MAE hat: 3.5681
Iter: 347, Loss: 429.5170, Train MAE hat: 3.5664
Iter: 348, Loss: 427.7227, Train MAE hat: 3.5647
Iter: 349, Loss: 425.9461, Train MAE hat: 3.5630
Iter: 350, Loss: 424.1877, Train MAE hat: 3.5613
Iter: 351, Loss: 422.4472, Train MAE hat: 3.5597
Iter: 352, Loss: 420.7247, Train MAE hat: 3.5580
Iter: 353, Loss: 419.0201, Train MAE hat: 3.5563
Iter: 354, Loss: 417.3335, Train MAE hat: 3.5546
Iter: 355, Loss: 415